In [1]:
import pandas as pd 
import numpy as np
import nltk
import re 
import os 
import random 
import matplotlib.pyplot as plt
import seaborn as sns 
import html
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import string

In [2]:
train = pd.read_csv("../input/nlp-getting-started/train.csv")
test = pd.read_csv("../input/nlp-getting-started/test.csv")


train.head()

sample = train["text"].values
target = train["target"].values

print(sample)
print(target)



In [3]:
train.shape, test.shape


In [4]:
#null값 확인
train.isnull().sum(),test.isnull().sum()

In [5]:
#null값을 0으로 채우기
train['location'] = train['location'].fillna(train['location'].mode()[0])
train['keyword'] = train['keyword'].fillna(train['keyword'].mode()[0])

test['location'] = test['location'].fillna(test['location'].mode()[0])
test['keyword'] = test['keyword'].fillna(test['keyword'].mode()[0])


In [6]:
#null값 확인
train.isnull().sum(),test.isnull().sum()

In [7]:
#create a new colums
train['processed_text'] = html.unescape(train['text'])
test['processed_text'] = html.unescape(test['text'])

In [8]:
def lookup_dict(text, dictionary):
    for word in text.split():
        if word.lower() in dictionary:
            if word.lower in text.split():
                text = text.replace(word, dictionary[word.lower()])
    return text

In [9]:
# Apostrophe Dictionary
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}


In [10]:
short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}


In [11]:
emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}


In [12]:
#disaster word dictionaty
disaster_dict = {
"ablaze": "ablaze disaster",
"accident": "accident disaster",
"accidents": "accidents disaster",
"aftershock": "aftershock disaster",
"ambulance": "ambulance disaster",
"ambulances": "ambulances disaster",
"annihilate": "annihilate disaster",
"annihilated": "annihilated disaster",
"annihilates": "annihilates disaster",
"annihilation": "annihilation disaster",
"apocalypse": "apocalypse disaster",
"armies": "armies disaster",
"army": "army disaster",
"armageddon": "armageddon disaster",
"arson": "arson disaster",
"arsonist": "arsonist disaster",
"attack": "attack disaster",
"attacked": "attacked disaster",
"attacks": "attacks disaster",
"avalanche": "avalanche disaster",
"bang": "bang disaster",
"battle": "battle disaster",
"bioterror": "bioterror disaster",
"bioterrorism": "bioterrorism disaster",
"bioterrorist": "bioterrorist disaster",
"blaze": "blaze disaster",
"blazed": "blazed disaster",
"blazen": "blazen disaster",
"blazes": "blazes disaster",
"blazing": "blazing disaster",
"bleed": "bleed disaster",
"bleeding": "bleeding disaster",
"blight": "blight disaster",
"blighted": "blighted disaster",
"blighting": "blighting disaster",
"blights": "blights disaster",
"blizzard": "blizzard disaster",
"blizzards": "blizzards disaster",
"blood": "blood disaster",
"bloody": "bloody disaster",
"bodybag": "bodybag disaster",
"bodybags": "bodybags disaster",
"bomb": "bomb disaster",
"bomber": "bomber disaster",
"bombing": "bombing disaster",
"bombs": "bombs disaster",
"burn": "burn disaster",
"burned": "burned disaster",
"burning": "burning disaster",
"burns": "burns disaster",
"burnt": "burnt disaster",
"casualties": "casualties disaster",
"casualty": "casualty disaster",
"catastrophe": "catastrophe disaster",
"catastrophic": "catastrophic disaster",
"chemical": "chemical disaster",
"chemicals": "chemicals disaster",
"cliff": "cliff disaster",
"cliffhanger": "cliffhanger disaster",
"cliffs": "cliffs disaster",
"collapse": "collapse disaster",
"collapsed": "collapsed disaster",
"collide": "collide disaster",
"collided": "collided disaster",
"collision": "collision disaster",
"crash": "crash disaster",
"crashed": "crashed disaster",
"crush": "crush disaster",
"crushed": "crushed disaster",
"curfew": "curfew disaster",
"cyclone": "cyclone disaster",
"danger": "danger disaster",
"dangerous": "dangerous disaster",
"dangerzone": "dangerzone disaster",
"dead": "dead disaster",
"death": "death disaster",
"demolish": "demolish disaster",
"demolished": "demolishes disaster",
"demolishes": "demolishes disaster",
"demolishing": "demolishing disaster",
"died": "died disaster",
"debris":  "debris disaster",
"derail": "derail disaster",
"derailed": "derailed disaster",
"destroy": "destroy disaster",
"destroyed": "destroyed disaster",
"destroying": "destroying disaster",
"destroys": "destroys disaster",
"derailment": "derailment disaster",
"detonate": "detonate disaster",
"detonated": "detonated disaster",
"detonates": "detonates disaster",
"detonating": "detonating disaster",
"detonation": "detonation disaster",
"devastated": "devastated disaster",
"devastation": "devastation disaster",
"damage": "damage disaster",
"damaged": "damaged disaster",
"damages": "damages disaster",
"deluge": "deluge disaster",
"deluged": "deluged disaster",
"demolish": "demolish disaster",
"demolished": "demolished disaster",
"demolition": "demolition disaster",
"desolate": "desolate disaster",
"destruction": "destruction disaster",
"disaster": "disaster disaster",
"disasters": "disasters disaster",
"displace": "displace disaster",
"displaced": "displaced disaster",
"drought": "drought disaster",
"drown": "drown disaster",
"drowned": "drowned disaster",
"drowning": "drowning disaster",
"drowns": "drowns disaster",
"earthquake": "earthquake disaster",
"earthquakes": "earthquake disaster",
"electrocute": "electrocute disaster",
"electrocuted": "electrocuted disaster",
"electrocutes": "electrocutes disaster",
"electrocuting": "electrocuting disaster",
"emergency": "emergency disaster",
"emergencies": "emergencies disaster",
"engulf": "engulf disaster",
"engulfed": "engulfed disaster",
"engulfing": "engulfing disaster",
"evacuate": "evacuate disaster",
"evacuated": "evacuated disaster",
"explode": "explode disaster",
"exploded": "exploded disaster",
"explodes": "explodes disaster",
"exploding": "exploding disaster",
"explosion": "explosion disaster",
"eyewitness": "eyewitness disaster",
"eyewitnesses": "eyewitnesses disaster",
"eyewitnessed": "eyewitnessed disaster",
"fail": "fail disaster",
"failed": "failed disaster",
"failure": "failure disaster",
"fall": "fall disaster",
"falling": "falling disaster",
"famine": "famine disaster",
"fatal": "fatal disaster",
"fatalities": "fatalities disaster",
"fatality": "fatality disaster",
"fear": "fear disaster",
"feared": "feared disaster",
"fearful": "fearful disaster",
"fears": "fears disaster",
"fearsome": "fearsome disaster",
"fell": "fell disaster",
"fire": "fire disaster",
"fires": "fires disaster",
"flame": "flame disaster",
"flames": "flames disaster",
"flaming": "flaming disaster",
"flood": "flood disaster",
"flooded": "flooded disaster",
"flooding": "flooding disaster",
"floods": "floods disaster",
"hail": "hail disaster",
"hailed": "hailed disaster",
"hailing": "hailing disaster",
"hails": "hails disaster",
"hailstorm": "hailstorm disaster",
"hailstormed": "hailstormed disaster",
"hailstorming": "hailstorming disaster",
"hailstorms": "hailstorms disaster",
"harm": "harm disaster",
"harmed": "harmed disaster",
"harming": "harming disaster",
"harms": "harms disaster",
"hazard": "hazard disaster",
"hazardous": "hazardous disaster",
"heatwave": "heatwave disaster",
"hell": "hell disaster",
"hellfire": "hellfire disaster",
"hijack": "hijack disaster",
"hijacked": "hijacked disaster",
"hijacker": "hijacker disaster",
"hijacking": "hijacking disaster",
"hostage": "hostage disaster",
"hostages": "hostages disaster",
"hurricane": "hurricane disaster",
"hurricanes": "hurricanes disaster",
"injure": "injure disaster",
"injured": "injured disaster",
"injures": "injures disaster",
"injuries": "injuries disaster",
"injuring": "injuring disaster",
"injury": "injury disaster",
"inundate": "inundate disaster",
"inundated": "inundated disaster",
"inundation": "inundation disaster",
"landslide": "landslide disaster",
"landslides": "landslides disaster",
"lava": "lava disaster",
"lightning": "lightning disaster",
"massacre": "massacre disaster",
"massacred": "massacred disaster",
"massacres": "massacres disaster",
"mayhem": "mayhem disaster",
"meltdown": "meltdown disaster",
"meltdowns": "meltdowns disaster",
"military": "military disaster",
"militarise": "militarise disaster",
"militarize": "militarize disaster",
"mudslide": "mudslide disaster",
"mudslides": "mudslides disaster",
"murder": "murder disaster",
"murdered": "murdered disaster",
"murderer": "murderer disaster",
"murderers": "murderers disaster",
"murders": "murders disaster",
"nuclear": "nuclear disaster",
"obliterate": "obliterate disaster",
"obliterated": "obliterated disaster",
"obliterates": "obliterates disaster",
"obliterating": "obliterating disaster",
"obliteration": "obliteration disaster",
"outbreak": "outbreak disaster",
"pandemonium": "pandemonium disaster",
"panic": "panic disaster",
"panicked": "panicked disaster",
"panicking": "panicking disaster",
"panics": "panics disaster",
"police": "police disaster",
"policeman": "policeman disaster",
"policemen": "policemen disaster",
"rainstorm": "rainstorm disaster",
"quarantine": "quarantine disaster",
"quarantined": "quarantined disaster",
"quarantines": "quarantines disaster",
"quarantining": "quarantining disaster",
"rainstormed": "rainstormed disaster",
"rainstorming": "rainstorming disaster",
"rainstorms": "rainstorms disaster",
"radiation": "radiation disaster",
"raze": "raze disaster",
"razed": "razed disaster",
"reactor": "reactor disaster",
"reactors": "reactors disaster",
"refugee": "refugee disaster",
"refugees": "refugees disaster",
"rescue": "rescue disaster",
"rescued": "rescued disaster",
"rescuer": "rescuer disaster",
"rescuing": "rescuing disaster",
"responder": "responder disaster",
"responders": "responders disaster",
"riot": "riot disaster",
"rioted": "rioted disaster",
"rioting": "rioting disaster",
"riots": "riots disaster",
"rubble": "rubble disaster",
"ruin": "ruin disaster",
"ruined": "ruined disaster",
"ruining": "ruining disaster",
"ruins": "ruins disaster",
"scream": "scream disaster",
"screamed": "screamed disaster",
"screaming": "screaming disaster",
"screams": "screams disaster",
"seismic": "seismic disaster",
"sank": "sank disaster",
"sink": "sink disaster",
"sinking": "sinking disaster",
"sunk": "sunk disaster",
"sinkhole": "sinkhole disaster",
"sinkholes": "sinkholes disaster",
"siren": "siren disaster",
"sirens": "sirens disaster",
"smoke": "smoke disaster",
"smoked": "smoked disaster",
"smoking": "smoking disaster",
"smokes": "smokes disaster",
"snow": "snow disaster",
"snowed": "snowed disaster",
"snowing": "snowing disaster",
"snows": "snows disaster",
"snowstorm": "snowstorm disaster",
"snowstormed": "snowstormed disaster",
"snowstorming": "snowstorming disaster",
"snowstorms": "snowstorms disaster",
"spill": "spill disaster",
"starvation": "starvation disaster",
"starve": "starve disaster",
"storm": "storm disaster",
"stormed": "stormed disaster",
"storming": "storming disaster",
"storms": "storms disaster",
"stretcher": "stretcher disaster",
"suicide": "suicide disaster",
"survive": "survive disaster",
"survived": "survived disaster",
"survives": "survives disaster",
"surviving": "surviving disaster",
"survivor": "survivor disaster",
"survivors": "survivors disaster",
"terror": "terror disaster",
"terrorism": "terrorism disaster",
"terrorist": "terrorist disaster",
"threat": "threat disaster",
"threaten": "threaten disaster",
"thunder": "thunder disaster",
"thundered": "thundered disaster",
"thundering": "thundering disaster",
"thunders": "thunders disaster",
"thunderstorm": "thunderstorm disaster",
"thunderstormed": "thunderstormed disaster",
"thunderstorming": "thunderstorming disaster",
"thunderstorms": "thunderstorms disaster",
"tornado": "tornado disaster",
"tornados": "tornados disaster",
"tragic": "tragic disaster",
"tragedy": "tragedy disaster",
"trap": "trap disaster",
"trapped": "trapped disaster",
"trapping": "trapping disaster",
"traps": "traps disaster",
"trauma": "trauma disaster",
"traumatise": "traumatise disaster",
"traumatised": "traumatised disaster",
"traumatises": "traumatises disaster",
"traumatising": "traumatising disaster",
"traumatize": "traumatize disaster",
"traumatized": "traumatized disaster",
"traumatizing": "traumatizing disaster",
"traumatizes": "traumatizes disaster",
"trouble": "trouble disaster",
"troubles": "troubles disaster",
"tsunami": "tsunami disaster",
"tsunamis": "tsunamis disaster",
"twister": "twister disaster",
"twisters": "twisters disaster",
"typhoon": "typhoon disaster",
"upheaval": "upheaval disaster",
"violence": "violence disaster",
"violent": "violent disaster",
"volcanic": "volcanic disaster",
"volcano": "volcano disaster",
"war": "war disaster",
"warred": "warred disaster",
"warring": "warring disaster",
"wars": "wars disaster",
"weapon": "weapon disaster",
"weaponize": "weaponize disaster",
"weaponise": "weaponise disaster",
"weapons": "weapons disaster",
"whirlwind": "whirlwind disaster",
"wildfire": "wildfire disaster",
"wildfires": "wildfires disaster",
"windstorm": "windstorm disaster",
"windstormed": "windstormed disaster",
"windstorming": "windstorming disaster",
"windstorms": "windstorms disaster",
"wound": "wound disaster",
"wounded": "wounded disaster",
"wounding": "wounding disaster",
"wounds": "wounds disaster",
"wreck": "wreck disaster",
"wreckage": "wreckage disaster",
"wrecked": "wrecked disaster",
}

In [13]:

train['processed_text'] = train['processed_text'].apply(lambda x: lookup_dict(x, apostrophe_dict))
test['processed_text'] = test['processed_text'].apply(lambda x: lookup_dict(x, apostrophe_dict))

train['processed_text'] = train['processed_text'].apply(lambda x: lookup_dict(x, short_word_dict))
test['processed_text'] = test['processed_text'].apply(lambda x: lookup_dict(x, short_word_dict))

train['processed_text'] = train['processed_text'].apply(lambda x: lookup_dict(x, emoticon_dict))
test['processed_text'] = test['processed_text'].apply(lambda x: lookup_dict(x, emoticon_dict))

train['processed_text'] = train['processed_text'].apply(lambda x: lookup_dict(x, disaster_dict))
test['processed_text'] = test['processed_text'].apply(lambda x: lookup_dict(x, disaster_dict))



In [14]:
#어간추출
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
stemmer = PorterStemmer()
words = stopwords.words("english")
#stemming
train['processed_text'] = train['text'].apply(lambda x: " ".join([stemmer.stem(i)                                                              
                                                                    for i in re.sub("[^a-zA-Z]"," ",x).split() if i not in words]).lower())
test['processed_text'] = test['text'].apply(lambda x: " ".join([stemmer.stem(i)
                                                                  for i in re.sub("[^a-zA-Z]"," ",x).split() if i not in words]).lower())


In [15]:
def clean_text(text):
    """Removes htmls, urls, special characters,numbers, and stopwords from the text.
    Also tokenizes the text, converts all the words into lower-case, and lemmatizes them, 
    before converting them back to sentences.""" 
    wnl = nltk.WordNetLemmatizer() #for lemmatizing
    stopword = nltk.corpus.stopwords.words('english')
    
    remove_html = BeautifulSoup(text, 'lxml').get_text() #removing html
    remove_url = re.sub(r"http\S+", "", remove_html)  #removing urls (replacing them with blanks)
    tokens = nltk.word_tokenize(remove_url)    #tokenizing the words
    lower = [word.lower() for word in tokens]  #converting words to lowercase
    remove_stopwords = [word for word in lower if word not in stopword]  #removing stopwords
    remove_char = [word for word in remove_stopwords if word.isalpha()]     #include only alphabets
    lemm_text = [wnl.lemmatize(word) for word in remove_char]     #lemmatizing the words
    cleaned_data = " ".join([str(word) for word in lemm_text])
    return cleaned_data

In [16]:
train['processed_text'] = train['text'].map(clean_text)
test['processed_text'] = test['text'].map(clean_text)
train['processed_text'].head()

In [17]:
#1번 초과로 나온 단어는 삭제
from collections import Counter
from itertools import chain

lists = train['processed_text'].str.split().tolist()
count = Counter(chain.from_iterable(lists))
train['processed_text'] = [' '.join([j for j in i if count[j]>1])for i in lists]

lists = test['processed_text'].str.split().tolist()
count = Counter(chain.from_iterable(lists))
test['processed_text'] = [' '.join([j for j in i if count[j]>1])for i in lists]



In [18]:
y = train.target
X = train['processed_text']
X_test = test['processed_text']


In [19]:
#model division
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify = y, test_size = 0.10, shuffle = True, random_state = 42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [20]:
#단어 가중치 계산
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_tfidf = TfidfVectorizer(stop_words = 'english', max_df = 0.7)
train_tfIdf = vectorizer_tfidf.fit_transform(X_train.values.astype('U'))
val_tfIdf = vectorizer_tfidf.transform(X_val.values.astype('U'))
X_test_tfIdf = vectorizer_tfidf.transform(X_test.values.astype('U'))


In [21]:
train_tfIdf.shape, val_tfIdf.shape, X_test_tfIdf.shape

In [22]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.metrics import roc_curve, auc, confusion_matrix, log_loss, f1_score
# max_depth = 6, 학습률은 0.1, 예제가 이진분류이므로 목적함수(objective)는 binary:logistic(이진 로지스틱)
# 오류함수의 평가성능지표는 logloss
# 부스팅 반복횟수는 400
# 조기중단을 위한 최소 반복횟수는 100

params = {'max_depth' : 6,
         'eta' : 0.1, 
         'objective' : 'binary:logistic',
         'eval_metric' : 'logloss',
         'early_stoppings' : 100 }

In [23]:
#임계점 찾기
def find_best_threshold(threshould, fpr, tpr):
    t = threshould[np.argmax(tpr*(1-fpr))]
    # (tpr*(1-fpr)) will be maximum if your fpr is very low and tpr is very high
    print("the maximum value of tpr*(1-fpr)", max(tpr*(1-fpr)), "for threshold", np.round(t,3))
    return t
#임계점 이상일 때 1, 미만일 때 0
def predict_with_best_t(proba, threshold):
    predictions = []
    for i in proba:
        if i>=threshold:
            predictions.append(1)
        else:
            predictions.append(0)
    return predictions

In [24]:
def plot_roc_curve(fpr_tr, tpr_tr):
    '''
    plot the ROC curve for the FPR and TPR value
    '''
    plt.plot(fpr_tr, tpr_tr, color='green', label='ROC_train AUC = {:0.2f} '.format(auc(fpr_tr, tpr_tr)))
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

In [25]:
d_train = xgb.DMatrix(train_tfIdf, label=y_train)
watchlist = [(d_train, 'train')]

bst = xgb.train(params, d_train, 400, watchlist, early_stopping_rounds=20, verbose_eval=10)

predict_y = bst.predict(d_train)
print("The test log loss is:",log_loss(y_train, predict_y, eps=1e-15))
#fpr,tpr,임계값
fpr_tfidf, tpr_tfidf, t_tfidf = roc_curve(y_train, predict_y)
print('F1 score',f1_score(y_train,predict_with_best_t(predict_y, find_best_threshold(t_tfidf,fpr_tfidf,tpr_tfidf))))
plot_roc_curve(fpr_tfidf,tpr_tfidf)

In [26]:
d_test = xgb.DMatrix(X_test_tfIdf)
y_ = bst.predict(d_test)
y_pred = predict_with_best_t(y_, find_best_threshold(t_tfidf,fpr_tfidf,tpr_tfidf))

In [27]:
submission = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
submission['target'] = y_pred
submission['id'] = test.id
submission.to_csv('submission.csv',index = False)
submission